In [34]:
import pandas as pd
import matplotlib.pyplot as plt

leak_data_raw =  pd.read_csv("Gasleak Data Sets/sensor_readings.csv")
leak_data_raw.columns = ["temp","Time","Sensor 1", "Sensor 2", "Sensor 3", "Sensor 4", "Sensor 5", "Sensor 6", "Sensor 7", "Sensor 8", "Sensor 9", "Sensor 10", "Sensor 11", "Sensor 12", "Sensor 13", "Sensor 14", "Sensor 15", "Sensor 16", "Sensor 17", "Sensor 18", "Sensor 19", "Sensor 20", "Sensor 21", "Sensor 22", "Sensor 23", "Sensor 24"]

weather_df = pd.read_csv("Gasleak Data Sets/weather_data.csv")
columns_to_average = ['Barometric_Pressure', 'Humidity', 'Temperature', 'Wind_Direction', 'Wind_Speed']
for col in columns_to_average:
    weather_df[col] = weather_df.groupby('timestamp')[col].transform('mean')
weather_df = weather_df.drop_duplicates()
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9

leak_rate_df = pd.read_csv("Gasleak Data Sets/leak_locations_and_rate.csv")


/var/folders/qt/r87857tx74n48x9wj7f9wljw0000gn/T/ipykernel_17157/2229401421.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9


In [35]:
weather_df.head()

,timestamp,Barometric_Pressure,Humidity,Temperature,Wind_Direction,Wind_Speed
0,1681776000,827.062090,31.740852,10.737401,320.568991,1.784585
60,1681776060,827.072498,31.765488,10.770846,309.686471,1.673587
120,1681776120,827.062251,31.582803,10.677768,294.296959,1.363441
180,1681776180,827.053569,31.905685,10.685998,275.853219,1.169298
240,1681776240,827.066506,31.705671,10.915012,276.878484,0.994745


In [36]:
idx = 0
leak_and_weather_data = leak_data_raw
t = leak_and_weather_data.at[idx,'time']
for index, row in weather_df.iterrows():
    

(1440, 6)